In [4]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import json
import re
import time
import sentencepiece
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, PegasusForConditionalGeneration, PegasusTokenizer, pipeline
import torch
random_seed=42

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
# df = pd.read_json(r'/content/drive/MyDrive/HIT/SHROOM/data/val.model-aware.v2.json',lines=False)
df = pd.read_json(r'/content/drive/MyDrive/HIT/SHROOM/data/test.model-aware.json',lines=False)

In [8]:
df_dm = df[df['task']=='DM']
df_dm.index = np.arange(df_dm.shape[0])

df_mt = df[df['task']=='MT']
df_mt.index = np.arange(df_mt.shape[0])

df_pg = df[df['task']=='PG']
df_pg.index = np.arange(df_pg.shape[0])

In [ ]:
# DM tgt ltg/flan-t5-definition-en-base
# MT either facebook/nllb-200-distilled-600M
# PG src tuner007/pegasus_paraphrase

DM

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

model = T5ForConditionalGeneration.from_pretrained("ltg/flan-t5-definition-en-base")
tokenizer = T5Tokenizer.from_pretrained("ltg/flan-t5-definition-en-base")
model.to(device)

def DM(input_text):

  inputs = tokenizer.encode_plus(input_text, return_tensors="pt")
  inputs = inputs.to(device)

  decoder_input_ids = torch.tensor([[tokenizer.pad_token_id]])
  decoder_input_ids = decoder_input_ids.to(device)

  token_probs = []
  token_entropies = []


  # Loop to generate each token
   for i in range(400): # Generate up to 400 tokens
       # Get the output of the model
       outputs = model(**inputs, decoder_input_ids=decoder_input_ids)

       # Get logits
       logits = outputs.logits

       #Convert logits to probabilities
       probs = torch.nn.functional.softmax(logits[:, -1, :], dim=-1)

       # Calculate entropy
       entropy = -torch.sum(probs * torch.log(probs), dim=-1)

       # Get the index with the highest probability
       _, predicted_index = torch.max(probs, dim=-1)

       #Convert index to text
       predicted_token = tokenizer.decode(predicted_index)

       # If the generated token is a terminator, then end the generation
       if predicted_token == tokenizer.eos_token:
           break

       # Add the generated token and its probability to the list
       token_probs.append(torch.max(probs).cpu().item())
       token_entropies.append(entropy.cpu().item())



       # Add the generated token to the input of the decoder
       decoder_input_ids = torch.cat([decoder_input_ids, predicted_index.unsqueeze(-1)], dim=-1)







  # Loop to generate each token
  for i in range(400):  # Generate up to 400 tokens
      # Get the output of the model
      outputs = model(**inputs, decoder_input_ids=decoder_input_ids)

      # 获取logits
      logits = outputs.logits

      # Convert logits to probabilities
      probs = torch.nn.functional.softmax(logits[:, -1, :], dim=-1)

      # Calculate entropy
      entropy = -torch.sum(probs * torch.log(probs), dim=-1)

      # Get the index with the highest probability
      _, predicted_index = torch.max(probs, dim=-1)

      # Convert index to text
      predicted_token = tokenizer.decode(predicted_index)

      # If the generated token is a terminator, then end the generation
      if predicted_token == tokenizer.eos_token:
          break

      # Add the generated token and its probability to the list
      token_probs.append(torch.max(probs).cpu().item())
      token_entropies.append(entropy.cpu().item())



      # Add the generated token to the input of the decoder
      decoder_input_ids = torch.cat([decoder_input_ids, predicted_index.unsqueeze(-1)], dim=-1)

  stats = {
        'token_prob_min': 1-np.min(token_probs),
        'token_prob_max': 1-np.max(token_probs),
        'token_prob_mean': 1-np.mean(token_probs),
        'token_entropy_min': np.min(token_entropies),
        'token_entropy_max': np.max(token_entropies),
        'token_entropy_mean': np.mean(token_entropies),
    }

  return stats


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def apply_DM(df):
    df_stats = df['src'].apply(DM).apply(pd.Series)
    for stat in ['token_prob_min', 'token_prob_max', 'token_prob_mean',
            'token_entropy_min', 'token_entropy_max', 'token_entropy_mean']:
        df[stat] = df_stats[stat]
    return df

df_dm = apply_DM(df_dm)

for stat in ['token_prob_min', 'token_prob_max', 'token_prob_mean']:
  print("\n\n****************", stat)
  for zhiding in np.arange(0.1,1,0.02):
    df_dm['label_pre'] = df_dm[stat].apply(lambda x: 1 if x>zhiding else 0)
    print('阈值：',round(zhiding,2),' 准确率：',round(accuracy_score(df_dm['label'], df_dm['label_pre']),2))
  print('Spearman：', round(spearmanr(df_dm[stat], df_dm['p(Hallucination)'])[0], 2))

for stat in ['token_entropy_min', 'token_entropy_max', 'token_entropy_mean']:
  print("\n\n****************", stat)
  for zhiding in np.arange(0.1,10,0.1):
    df_dm['label_pre'] = df_dm[stat].apply(lambda x: 1 if x>zhiding else 0)
    print('阈值：',round(zhiding,2),' 准确率：',round(accuracy_score(df_dm['label'], df_dm['label_pre']),2))
  print('Spearman：', round(spearmanr(df_dm[stat], df_dm['p(Hallucination)'])[0], 2))

In [10]:
def calibrate_probability(p, p_value):
    if p == p_value:
        return 0.5
    elif p > p_value:
        return 0.5 + (p - p_value) * (1 - 0.5) / (1 - p_value)
    else:  # p < p_value
        return 0 + (p - 0) * (0.5 - 0) / (p_value - 0)

In [11]:
def calibrate_probability_entropy(p, p_value, max):
    if p == p_value:
        return 0.5
    elif p >= max:
        return 1
    elif p > p_value:
        return 0.5 + (p - p_value) * (1 - 0.5) / (max - p_value)
    else:  # p < p_value
        return 0 + (p - 0) * (0.5 - 0) / (p_value - 0)

In [12]:
df_all1 = pd.DataFrame()

In [ ]:
# token_prob_min
# Threshold: 0.76 Accuracy: 0.63**
# token_prob_max
# Threshold: 0.26 Accuracy: 0.54
# token_prob_mean
# Threshold: 0.3 Accuracy: 0.59**
# token_entropy_min
# Threshold: 0.8 Accuracy: 0.54
# token_entropy_max
# Threshold: 3.8 Accuracy: 0.64**
# token_entropy_mean
# Threshold: 1.7 Accuracy: 0.64**

# token_entropy_mean is selected based on the effect of the verification set.

In [13]:
def apply_DM(df):
    df_stats = df['src'].apply(DM).apply(pd.Series)
    for stat in ['token_prob_min', 'token_prob_max', 'token_prob_mean',
            'token_entropy_min', 'token_entropy_max', 'token_entropy_mean']:
        df[stat] = df_stats[stat]
    return df

df_dm = apply_DM(df_dm)

In [16]:
df_dm['token_entropy_mean'].describe(percentiles=[.98])

count    554.000000
mean       1.711624
std        0.580928
min        0.300135
50%        1.738078
98%        2.724807
max        4.734089
Name: token_entropy_mean, dtype: float64

In [17]:
p_value1 = 1.7
max = 2.7
df_dm['pred'] = df_dm['token_entropy_mean'].apply(lambda x: calibrate_probability_entropy(x, p_value1, max))

In [20]:
df_dm.columns

Index(['id', 'src', 'tgt', 'hyp', 'task', 'model', 'token_prob_min',
       'token_prob_max', 'token_prob_mean', 'token_entropy_min',
       'token_entropy_max', 'token_entropy_mean', 'pred'],
      dtype='object')

In [21]:
df_dm = df_dm[['id', 'src', 'tgt', 'hyp', 'task', 'model',  'pred']]

In [22]:
df_all1 = pd.concat([df_all1, df_dm])

In [23]:
df_all1.columns

Index(['id', 'src', 'tgt', 'hyp', 'task', 'model', 'pred'], dtype='object')

In [24]:
del model, tokenizer
torch.cuda.empty_cache()

MT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/nllb-200-distilled-600M"
# tokenizer1 = AutoTokenizer.from_pretrained(model_name, src_lang="zho_Hans")
tokenizer1 = AutoTokenizer.from_pretrained(model_name, src_lang="eng_Latn")
tokenizer2 = AutoTokenizer.from_pretrained(model_name, src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)

In [26]:
def MT(input_text):
    inputs = tokenizer1.encode(input_text, return_tensors='pt')
    inputs = inputs.to(device)
    outputs = model.generate(inputs, forced_bos_token_id=tokenizer1.lang_code_to_id["eng_Latn"], output_scores=True, return_dict_in_generate=True)

    token_probs = []
    token_entropies = []
    for i in np.arange(1,len(outputs['scores'])-1):
        probs = outputs['scores'][i].softmax(dim=-1)[0]
        token_probs.append(torch.max(probs).item())
        token_entropies.append(-torch.sum(probs * torch.log(probs)).item())

    stats = {
        'token_prob_min': 1-np.min(token_probs),
        'token_prob_max': 1-np.max(token_probs),
        'token_prob_mean': 1-np.mean(token_probs),
        'token_entropy_min': np.min(token_entropies),
        'token_entropy_max': np.max(token_entropies),
        'token_entropy_mean': np.mean(token_entropies),
    }

    return stats

In [18]:
MT("请介绍你自己")
# prob:min mean
# entropy:max mean

{'token_prob_min': 0.4403417706489563,
 'token_prob_max': 0.18828970193862915,
 'token_prob_mean': 0.3134673237800598,
 'token_entropy_min': 1.7458938360214233,
 'token_entropy_max': 3.8632514476776123,
 'token_entropy_mean': 2.651001731554667}

In [ ]:
def apply_MT(df):
    df_stats = df['src'].apply(MT)
    for stat in ['token_prob_min', 'token_prob_max', 'token_prob_mean', 'token_entropy_min', 'token_entropy_max', 'token_entropy_mean']:
        df[stat] = df_stats.apply(lambda x: x[stat])
    return df

df_mt = apply_MT(df_mt)

# model prediction
for stat in ['token_prob_min', 'token_prob_max', 'token_prob_mean']:
  print("\n\n****************", stat)
  for zhiding in np.arange(0.1,1,0.02):
    df_mt['label_pre'] = df_mt[stat].apply(lambda x: 1 if x>zhiding else 0)
    print('阈值：',round(zhiding,2),' 准确率：',round(accuracy_score(df_mt['label'], df_mt['label_pre']),2))
  print('Spearman：', round(spearmanr(df_mt[stat], df_mt['p(Hallucination)'])[0], 2))

for stat in ['token_entropy_min', 'token_entropy_max', 'token_entropy_mean']:
  print("\n\n****************", stat)
  for zhiding in np.arange(0.1,10,0.1):
    df_mt['label_pre'] = df_mt[stat].apply(lambda x: 1 if x>zhiding else 0)
    print('阈值：',round(zhiding,2),' 准确率：',round(accuracy_score(df_mt['label'], df_mt['label_pre']),2))
  print('Spearman：', round(spearmanr(df_mt[stat], df_mt['p(Hallucination)'])[0], 2))

In [ ]:
# token_prob_min
# Threshold： 0.88  Accuracy： 0.62
# token_prob_max
# Threshold： 0.1  Accuracy： 0.59
# token_prob_mean
# Threshold： 0.46  Accuracy： 0.62
# token_entropy_min
# Threshold： 1.0  Accuracy： 0.6
# token_entropy_max
# Threshold： 7.3  Accuracy： 0.61
# token_entropy_mean
# Threshold： 3.4  Accuracy： 0.64 **
# token_entropy_mean is selected based on the effect of the verification set.

In [27]:
def apply_MT(df):
    df_stats = df['src'].apply(MT).apply(pd.Series)
    for stat in ['token_prob_min', 'token_prob_max', 'token_prob_mean',
            'token_entropy_min', 'token_entropy_max', 'token_entropy_mean']:
        df[stat] = df_stats[stat]
    return df

df_mt = apply_MT(df_mt)

In [28]:
df_mt['token_entropy_mean'].describe(percentiles=[.98])

count    563.000000
mean       3.485392
std        0.971581
min        0.941815
50%        3.432197
98%        5.509425
max        5.787091
Name: token_entropy_mean, dtype: float64

In [29]:
p_value1 = 3.4
max = 5.5
df_mt['pred'] = df_mt['token_entropy_mean'].apply(lambda x: calibrate_probability_entropy(x ,p_value1, max))

In [30]:
df_mt = df_mt[['id', 'src', 'tgt', 'hyp', 'task', 'model',  'pred']]

In [31]:
df_all1 = pd.concat([df_all1, df_mt])

In [34]:
df_all1.shape

(1125, 7)

In [32]:
del model, tokenizer1, tokenizer2
torch.cuda.empty_cache()

PG

In [35]:
from openai import OpenAI
import numpy as np

client = OpenAI()

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'

tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

def get_response(input_text, num_return_sequences, num_beams):
    batch = tokenizer([input_text], truncation=True, padding='longest', max_length=200, return_tensors="pt").to(device)
    translated = model.generate(**batch, max_length=200, num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text, translated

In [57]:
def PG(context):
  while True:
    try:
      print(1)
      num_beams = 5
      num_return_sequences = 5
      responses, translated = get_response(context, num_return_sequences, num_beams)
      answer = []
      for response in responses:

        content = f"Context: {response}\n\Sentence: {context}\n\Is the sentence supported by the context above?\n\Answer Yes or No:"

        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "user", "content": content}],
            temperature=0,
            max_tokens=10,
            n=1
        )

        for i in np.arange(len(response.choices)):
          if 'yes' in response.choices[i].message.content.lower():
            answer.append(0)
          elif 'no' in response.choices[i].message.content.lower():
            answer.append(1)
          else:
            print(i, '***', response.choices[i].message.content.lower())
            answer.append(1)
          # print(response.choices[i].message.content)

      time.sleep(0.3)
      return np.mean(answer)
    except Exception as e:
      print(e)
      time.sleep(5)


In [56]:
PG("hello")

1


0.0

In [ ]:
df_pg['pred'] = df_pg['src'].apply(PG)

In [ ]:
num_beams = 5
num_return_sequences = 5
context = "The ultimate test of your knowledge is your capacity to convey it to another."
response, translated = get_response(context,num_return_sequences,num_beams)
response

In [59]:
df_pg = df_pg[['id', 'src', 'tgt', 'hyp', 'task', 'model',  'pred']]

In [60]:
df_all1 = pd.concat([df_all1, df_pg])

In [61]:
# final
df_all1 = df_all1.sort_values('id')
df_all1.index=np.arange(df_all1.shape[0])

In [62]:
# model result
df_all1['p(Hallucination)'] = df_all1['pred']
df_all1['label'] = (df_all1['pred']>=0.5).astype(int).tolist()

test_data_aware_all = df_all1

path_val_model_aware_output = "test.model-aware.json"
output_json = []
for i in np.arange(test_data_aware_all.shape[0]):
    output_label = 'Hallucination' if test_data_aware_all.loc[i,'label'] == 1 else 'Not Hallucination'
    prob=test_data_aware_all.loc[i,'p(Hallucination)']
    id=test_data_aware_all.loc[i,'id']
    item_to_json = {"label":output_label, "p(Hallucination)":np.float64(prob), "id":int(id)}
    output_json.append(item_to_json)
f = open(path_val_model_aware_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()